In [2]:
import math
import numpy as np
import pandas as pd
import datetime as dt
import scipy.stats as si

In [39]:
S0 = 100
K = 98
T = [0,0.25,0.5,0.75,1]
dt = 0.25
r = 0
delta = 0.02
#q1 sigma cst = 23%
sigma = 0.23
mcpath = 1000
n = len(T)
#matrix N*5
np.random.seed( 420 ) 

In [40]:
vol = 0.23
var = vol ** 2
dailyVol = np.sqrt( var / dt )
time=1
dt=0.25
inter = time * dt

In [41]:
S = np.zeros((mcpath,n))

In [42]:
for i in range(0,mcpath):
    t = 0 
    W = 0 
    C=np.ones(5)
    for j in range(0,n):
        if j==0:
            C[j]=100
        else:
            W = np.random.normal(0,np.sqrt(dt))
            C[j]=np.exp((r-delta-0.5*sigma**2)*dt+sigma*W)
        S[i,j] = np.prod(C)

In [43]:
# Compute A for t1 t2 t3 t3
A = np.zeros((mcpath,n-1))
A[:,0] = S[:,1]
A[:,1] = (S[:,1] + S[:,2])/2
A[:,2] = (S[:,1] + S[:,2] + S[:,3])/3
A[:,3] = (S[:,1] + S[:,2] + S[:,3] + S[:,4])/4

In [44]:
#def function to do the intrinsic value
def intr(a):
    res = np.array([])
    for k in range(0,len(a)):
        if a[k] > 0:
            res = np.append(res,a[k])
        else:
            res = np.append(res,0)
    return res

In [46]:
C = np.zeros((mcpath,n))
C[:,n-1] = intr(S[:,n-1]-K)
for i in range(1,n):
    X = np.array([np.ones(mcpath),S[:,n-1-i],S[:,n-1-i]**2,S[:,n-1-i]**3,A[:,n-1-i],A[:,n-1-i]**2,A[:,n-1-i]**3]).T
    Y = np.sum(C,axis = 1) 
    beta = np.dot(np.linalg.inv(np.dot(X.T,X)),np.dot(X.T,Y))
    Y_reg = np.dot(X,beta)
    stop = intr(S[:,n-1-i]-K) > Y_reg
    C[:,n-1-i] = intr(S[:,n-1-i]-K)*stop
    for j in range(0,mcpath):
        if (stop[j] == True):  
            if (C[j,n-i-1] > (C[j,n-i])) and (stop[j] == True):
                C[j,n-i:n] = 0


In [47]:
pd.DataFrame(C) 

,0,1,2,3,4
0,0.0,0.00000,0.000000,0.000000,0.000000
1,0.0,0.00000,15.058820,0.000000,0.000000
2,0.0,0.00000,39.863161,0.000000,0.000000
3,0.0,0.00000,0.000000,0.000000,0.000000
4,0.0,0.00000,0.000000,0.000000,11.674056
...,...,...,...,...,...
995,0.0,0.00000,0.000000,36.257348,0.000000
996,0.0,0.00000,0.000000,0.000000,0.000000
997,0.0,0.00000,0.000000,30.627580,0.000000
998,0.0,12.77337,0.000000,0.000000,0.000000


In [50]:
price = sum(sum(C))/mcpath
price

13.627884653033096

In [111]:
def bs(S, K, T, r, q, sigma):
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    call = (S * np.exp(-q * T) * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    
    return call

In [112]:
bs(S0, K, 1,r,delta, sigma)

8.983224650185882

## point b

In [61]:
ind= pd.read_csv('/Users/francescomaizza/Documents/GitHub/advanced_derivatives/problem_set_7/strikes.csv',names=['s'])/2772.70

In [62]:
mat=pd.read_csv('problem_set_7/maturities.csv')

In [63]:
v=list(mat.columns)

In [64]:
vol= pd.read_csv('/Users/francescomaizza/Documents/GitHub/advanced_derivatives/problem_set_7/impv_total.csv',names=v)
vol.index=ind['s']

In [65]:
vol

,0.025,0.101,0.197,0.274,0.523,0.772,1,1.5,1.769
s,,,,,,,,,
0.40,8.287686e-01,8.170852e-17,3.853516e-16,3.120542e-16,5.890001e-16,0.157089,0.140222,3.284022e-16,0.102220
0.41,9.399773e-01,6.161537e-01,5.158837e-01,4.910442e-01,4.100280e-01,0.368878,0.343337,3.058936e-01,0.289128
0.42,9.329367e-01,6.189532e-01,5.233756e-01,5.027064e-01,4.255322e-01,0.386681,0.362494,3.268166e-01,0.310087
0.43,9.200532e-01,6.148949e-01,5.232228e-01,5.055190e-01,4.318259e-01,0.395100,0.372230,3.383986e-01,0.321951
0.44,9.053031e-01,6.082117e-01,5.199562e-01,5.045818e-01,4.340538e-01,0.399283,0.377658,3.456183e-01,0.329547
...,...,...,...,...,...,...,...,...,...
1.96,5.418673e-01,2.746558e-01,2.364537e-01,2.094809e-01,1.845412e-01,0.168413,0.157601,1.569358e-01,0.165017
1.97,4.206551e-16,2.798652e-01,2.362566e-01,2.091682e-01,1.837777e-01,0.167374,0.156402,1.549458e-01,0.162194
1.98,5.520387e-01,2.815722e-01,2.355584e-01,2.083782e-01,1.823776e-01,0.165643,0.154487,1.519794e-01,0.158148
